# Machine Learning for Graphs - Tutorial A: Triple Scoring with DistMult

*Fill in your names and group number here:*

**NAME STUDENT A** :  

**NAME STUDENT B** :  

**GROUP NUMBER** :  



Implementing a machine learning experiment with graph data is an important skill that you will learn as part of this course. This hands-on tutorial will help you develop this skill, as well as help you familiarize yourself with many of the steps and techniques that you will likely need to use for your final project.

The oldest and arguably most well-studied machine learning task for graphs is that of *link prediction*. The goal of this task is, as the name implies, to predict the existence of edges in a graph. Since real-world graphs often have large gaps in the information they encode, represented by missing links, and because this task has the potential of predicting these missing links, link prediction is also known as *knowledge base completion*.

For this tutorial, you are asked to implement a link prediction model. Once correctly implemented, you are to train and test the model on a graph-based dataset that you have prepared during the first half of the tutorial. To help you on your way, we have already prepared this Python Notebook.

You are asked to team up with another student and to work together on this tutorial. Please register your team by creating a new group and by adding both members.

---

## Working with Graphs

Most major programming languages do not support graph data out of the box. Therefore, before we can continue, we first need to download and install the [RDFlib package](https://rdflib.readthedocs.io/en/stable/), which extends Python with an interface to graphs that are encoded using the *Resource Description Format* (RDF). The RDF data model is a W3C open standard to encode knowledge, information, and data in graph form, and is a common preferred choice for creating *knowledge graphs*. Rather than using the common prefix notation for links, i.e., `relation(head, tail)`, RDF knowledge graphs use the *in-fix* notation: `(head, relation, tail)`. Because of this, the links in a knowledge graph are often also called *triples*.

**Run the cells below to install the RDFlib package in your Python environment and to import and inspect the graph**

In [ ]:
# install the rdflib package
%pip install rdflib

In [ ]:
import gzip
import rdflib

In [ ]:
# read the data from disk

g_train = rdflib.Graph()
g_test = rdflib.Graph()

path = './data/'
with gzip.open(path + f'wn18rr_train.nt.gz', 'r') as gf:
    g_train.parse(data = gf.read(), format = 'nt')
with gzip.open(path + f'wn18rr_test.nt.gz', 'r') as gf:
    g_test.parse(data = gf.read(), format = 'nt')

In [ ]:
# test whether reading the data was successful by printing 10 links from the train and test set each

print('TRAIN:', end='\n')
for h,r,t in list(g_train)[:10]:
    print(f'{h} {r} {t}', end='\n')

print('TEST:', end='\n')
for h,r,t in list(g_test)[:10]:
    print(f'{h} {r} {t}', end='\n')

### PyTorch

In this course we will make use of the [*PyTorch*](https://pytorch.org/) machine learning package, which provides a large array of convenient functions and tools to implement and run machine learning experiments. The core data structure in PyTorch is the *tensor*. Taking a broad definition of the tensor, it is used in PyTorch to store almost everything from scalars and vectors to matrices and higher-order tensors.

Much of your time will be spend working with this package.

**Run the cells below to install the PyTorch package in your Python environment and to set a manual seed**

In [ ]:
# install pytorch
%pip install torch

In [ ]:
import torch
import torch.nn as nn

seed = 42
torch.manual_seed(seed)  # allow for reproducability

## Data Preparation

Knowledge graphs are an extension of *labeled digraphs* and, as such, have labeled arcs and nodes. In case of the arcs, these labels represent relationship types, which typically occur more than once. In other words, arcs with the same label convey the same relationship. The situation is more complex for the nodes: some of the nodes represent *entities* (unique 'things', tangible or otherwise) which are labeled using the *Universal Resource Identifier* (URI). Examples of entities are people, objects, concepts, and dreams. Other nodes represent *literals* (raw data values), such as numbers, dates, and strings.

Dealing with these different labels can be challenging. It is therefore common to simplify the knowledge graph to a regular digraph. This is done by first enumerating all unique labels, and by then re-encoding the graph using these numbers. This is done separately for the nodes and the relations.

### Task 1: Encode the graph using incides

Write a procedure to transform the graph to an integer-encoding. For example, given a mapping `'maried' = 0`, `'age' = 5`, `'25' = 66`,`'john' = 43`, and `'kate' = 187`, the triples `(john, married, kate)` and `(john, age, 25)` should become `[43, 0, 187]` and `[43, 5, 66]`. Ensure that the encoding of the training set corresponds with that of the test set. The result should be two tensors and four maps.

In [ ]:
# generate mapping tables

i2n = set()  # integers to nodes map
i2r = set()  # integers to relation map

# your code here
# r2i = ...
# n2i = ...
# ...

num_nodes = len(i2n)
num_relations = len(i2r)

In [ ]:
# convert the graphs to an integer-encoding tensor

data_train = torch.zeros((len(g_train), 3), dtype=int)
# ...

data_test = 

Run the cell below to test your procedures

In [ ]:
# Test your procedure.

print('TRAIN:', end='\n')
for triple in data_train[:10]:
    print(triple, end='\n')

print('TEST:', end='\n')
for triple in data_test[:10]:
    print(triple, end='\n')

## A simple triple scoring model

Under the hood, a typical link prediction workflow defines a mathematical function f that takes a link as input and then returns the score of that link, or *triple*. The higher the score, the more likely the link is thought to exist. Given a set of candidate links, we can therefore compute all their scores and rank them in descending order. The top-k links are then the most likely to exist according to our model. Of course, to get reliable scores our model first needs to learn how to recognize possible valid links.

There are various ways for a link prediction model to learn how to distinguish between valid and invalid links. Some models use a parameterised scoring function with a set of weights that it updates every epoch, while treating the links' vector representations as immutable objects that have been initialized before training, as is common with traditional machine learning. Other methods, such as the one you will implement today, approach the problem in a different way, by optimizing the links' vector representations themselves. In a sense, the vector representations act as their own weights. 

### DistMult

Around 2015, a group of researchers of the Cornell University in the USA published a [paper](https://arxiv.org/abs/1412.6575) [1] in which they introduced a simple bilinear triple scoring function which performed surprisingly well on link prediction tasks. This scoring function, called *DistMult*, is defined as

$$ y^T_{e1} M_r y_{e2} \quad\quad\text{(formula 2 in [1])}$$

with $y_{i}$ the internal representation belonging to node $i$, and with $M_r$ the diagonal matrix representation for relationship type $r$. Both these representations are updated each iteration by optimizing for a strong separation between positive and negative samples.

    
*[1] Yang, B., Yih, S. W. T., He, X., Gao, J., & Deng, L. (2015). Embedding Entities and Relations for Learning and Inference in Knowledge Bases. In Proceedings of the International Conference on Learning Representations (ICLR) 2015.*

### Task 2: Implement DistMult

Implement DistMult as a subclass of the PyTorch `nn.Module` class. Specifically, this entails writing the initialisation function `__init__()` and a forward function `forward()`. Make it possible to specify the size of the dimensions used for the various vector representations. Test your model afterwards on a small subset of the training set.

In [ ]:
class DistMult(nn.Module):
    def __init__(self, ...):
        """ DistMult
        """
        super().__init__()

        # your code here

    def forward(self, X:torch.Tensor):
        # your code here

Run the following cell to initialize and test your implementation

In [ ]:
model = DistMult(...)

samples = data_train[:10]
out = model(samples)

print(out)

## Negative Sampling

A common approach for training link prediction models is to present the model with a set of positive and negative samples. Here, the positive samples are links that are known to exist in the graph, whereras the negative samples are links that are known never to exist in the graph. In practice, the set of positive samples is often just a subset of the graph, while the set of negative samples is generated by corrupting the existing links using some strategy. This is called *negative sampling*. By labeling the positive and negative samples accordingly, and by updating the weights on how these samples are scored, link prediction models learn to rank candidate links accurately.


### Task 3a: Corrupting triples

Write a procedure to generate negative samples. The procedure should return a new tensor with non-existing triples. Use Python comments to explain the strategy that you decided on, and test your procedure on a small subset of the data afterwards.

In [ ]:
def corrupt_triples(...):
    # your code here

    return data_corrupt

Run the following cell to test your procedure.

In [ ]:
samples = data_train[:10]
print(f"original data:\n{samples}\n")

samples_corrupted = corrupt_triples(...)  # add your parameters
print(f"corrupted data:\n{samples_corrupted}")

## Mini Batching

Real-world graphs are often quite large, which translates in a large number of links. It is, for example, not uncommon for a graph to have many millions of links. Just think about how many people are subscribed to Facebook, how many others they are connected to, and how many attributes an average user has associated with their account. Learning over so many links in a single go is often impossible, especially if we want to speed up our computations using a GPU, which has a limited amount of on-device memory.

Mini batching is a technique that enables you to learn on partitions of the data. This is typically achieved by splitting the data into roughly even parts, and by updating the model after each part. Next to alleviating scalibity issues, updating the model after seeing a subset of the data has the additional benefit of creating a smoother convergence, since any sudden changes in gradient by individual samples is compensated by other samples in the batch.

### Task 3b: Creating batches

Write a procedure to generate mini batches of triples. The procedure should return a list of batches. Take into account the overhead that copying batches to GPU has by distributing the triples in some sensible way. The preferred batch size must be a parameter.

In [ ]:
def mk_batches(...) -> list:
    # your code here
    return batches

Run the following cell to test your function

In [ ]:
for i, batch in enumerate(mk_batches(...), 1):  # add your parameters
    print(f"batch {i}: {batch}")

## Training and testing

Training and testing are two important steps in the experimental workflow. Here, it is important to remember that the test data must be used exactly once, for testing, and that the test run must use a predetermined set of hyperparameter values. If this were not the case, then you are simply optimizing for improved performance on the test split rather than on the entire dataset. 

While left out here, it is also often good practice to create a validation set. The validation set can be used for hyperparameter optimization by training the model with various different hyperparameter values and by evaluating the performance on the validation set.

### Task 4a: The training loop

Write a procedure to train the model. Specifically, create a loop that passes the entire training set through the model every epoch, while computing the loss and updating the weights after each batch. Ensure that your batches and negative samples get randomized or regenerated each epoch. Use the Adam optimizer and the BCE with logits loss.

In [ ]:
# set hyperparameters
learning_rate = ...
num_epoch = 50
corrupt_probability = ...
batch_size = ...

# set optimizer and loss function
optimizer = ...
loss_function = ...

num_train = data_train.shape[0]
for epoch in range(1, num_epoch+1):
    print(f'Epoch {epoch:3d} - ', end='')

    # create batches
    batches = ...         # your code here
    num_batches = len(batches)

    batch_loss_tensor = torch.zeros(num_batches, dtype=torch.float32)
    for batch_id, batch in enumerate(batches):
        data_batch = ...
        num_samples = data_batch.shape[0]

        # create negative samples by randomly assigning random node indices in the object position
        data_batch_corrupt = ...
        ...
 
        # create labels; positive samples are 1, negative 0
        Y = ... 

        # allow model parameters to be learned   
        model.train()         

        # compute scores for positive and negative triples  
        Y_hat = ...
        
        # compute loss
        batch_loss = ...
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        batch_loss = float(batch_loss)  # release memory of computation graph
        batch_loss_tensor[batch_id] = batch_loss

    print(f'loss on train set: {batch_loss_tensor.mean():0.4f}')

### Task 4b: The test loop

Write a procedure to test the now-trained model. Specifically, create a loop that passes the entire test set through the model every epoch, while computing the loss after each batch. Ensure that the weights of your model are frozen during testing.

In [ ]:
num_test = data_test.shape[0]

batch_size = ...
batches = ...
num_batches = len(batches)

batch_loss_tensor = torch.zeros(num_batches, dtype=torch.float32)
for batch_id, batch in enumerate(batches):
    data_batch = ...
    num_samples = data_batch.shape[0]

    # create negative samples by randomly assigning random node indices in the object position
    data_batch_corrupt = ...

    # create labels; positive samples are 1, negative 0
    Y = ...

    # freeze model parameters for evaluation
    model.eval()
    
    # compute scores for positive and negative triples  
    Y_hat = ...

    # compute loss
    batch_loss = ...

    batch_loss = float(batch_loss)  # release memory of computation graph
    batch_loss_tensor[batch_id] = batch_loss

    # compute hits@k (see task 5)
    hits_at_k = ...

print(f'loss on test set: {batch_loss_tensor.mean():0.4f}')
for k,v in hits_at_k.items():
    print(f'hits@{k}: {v:0.4f}')

### Evaluation

Two different metrics are commonly used to evaluate link prediction models: the *mean reciprocal rank* (MRR) and *hits@k*. The MRR equals the arithmetic mean of reciprocal ranks, and is defined as $$ MRR = \frac{1}{|\mathcal{T}|} \displaystyle\sum_{t \in \mathcal{T}} r(t)^{-1}$$

with $\mathcal{T}$ the set of positive triples and $r(t)$ the rank of triple $t$. The MRR returns a score in $(0, 1]$ where higher is better.

The *hits@k* equals the fraction of positive samples that appear in the first $k$ entries of the sorted rank list. The metric is similar to the MRR, yet cheaper to compute and more sensitive to irregularities. The *hits@k* is formaly defined as $$\text{hits@k} = \frac{1}{|\mathcal{T}|} \displaystyle\sum_{t \in \mathcal{T}} 1 \text{ iff } r(t) \le k$$

Common values for $k$ are 1, 3, 5, and 10. The returned score lie between $(0, 1]$ where higher is better.

### Task 5: Evaluate the model

Write a procedure to calculate the *hits@k* metric and update task 4b to output the average *hits@k* score for $k \in \{1, 3, 10\}$. Ensure that the score equals $1.0$ if all top-$k$ entries are positive samples. Be aware that you will need to rerun the test loop.

In [ ]:
def compute_hits_at_k(...) -> float:
     # your code here

## Deliverable

Once you and your team member are satisfied with the results, you are to rerun the notebook from scratch and to export it to PDF format. This cleans up the output and makes it easier for us to provide feedback. Specifically, first select Restart kernel and run all cells followed by Save and export Notebook as -> PDF.

**Please rename the PDF file to `ML4G-PA1_GROUP<groupID>.pdf` and submit the file before noon the next Monday**